In [1]:
import pydgraph

client_stub = pydgraph.DgraphClientStub("localhost:9080")
client = pydgraph.DgraphClient(client_stub)

def run(query, vars=None):
    txn = client.txn(read_only=True)
    res = txn.query(query, variables=vars or {})
    return res.json

In [2]:
import networkx as nx
from pyvis.network import Network
from IPython.display import display, HTML
import json # You need this if your dgraph_res is a JSON string

def show_graph(dgraph_res):
    # --- Data Processing (No changes needed here) ---
    G = nx.DiGraph()
    dgraph_res = json.loads(dgraph_res)
    data = dgraph_res.get('q', [])

    for parent in data:
        p_uid = parent.get('uid')
        p_label = parent.get('label', p_uid)
        G.add_node(p_uid, label=p_label, title=f"ID: {p_uid}", color="#ff5733", size=25)
        
        relationships = parent.get('rel', [])
        for child in relationships:
            c_uid = child.get('uid')
            c_label = child.get('label', c_uid)
            c_uri = child.get('uri', 'No URI')
            G.add_node(c_uid, label=c_label, title=f"URI: {c_uri}", color="#33C4FF")
            G.add_edge(p_uid, c_uid)
    # -----------------------------------------------

    net = Network(
        notebook=True, 
        height="500px", 
        width="100%", 
        bgcolor="#222222", 
        font_color="white", 
        cdn_resources='in_line' # Ensures all resources are bundled
    )

    net.from_nx(G)

    # =======================================================
    # FIX: Choose ONE of the following display options.
    # =======================================================

    # OPTION 1 (Recommended): Use net.show()
    # If notebook=True, net.show() handles HTML injection automatically 
    # and saves the file for persistence.
    net.show("networkx-pyvis.html")


In [7]:
query = """
query all($id: string) {
  q(func: uid($id)) {
    uid
    label
    rel_label
    rel @facets {
      uid
      uri
      label
    }
  }
}
"""
res = run(query, {"$id": "0x123"})

res



b'{"q":[{"uid":"0x123","label":"dump","rel_label":"etymologically related to","rel":[{"uid":"0x190","uri":"/c/en/discharge","label":"discharge","rel|edge_id":"/c/en/dump-/r/Synonym-/c/en/discharge-0000","rel|label":"synonym","rel|relation":"/r/Synonym","rel|sentence":"","rel|source":"RG"},{"uid":"0x325da","uri":"/c/en/sty","label":"sty","rel|edge_id":"/c/en/dump-/r/Synonym-/c/en/sty-0000","rel|label":"synonym","rel|relation":"/r/Synonym","rel|sentence":"","rel|source":"RG"},{"uid":"0x4a1ae","uri":"/c/en/junk","label":"junk","rel|edge_id":"/c/en/dump-/r/Synonym-/c/en/junk-0000","rel|label":"synonym","rel|relation":"/r/Synonym","rel|sentence":"","rel|source":"RG"},{"uid":"0x61d17","uri":"/c/en/hole","label":"hole","rel|edge_id":"/c/en/dump-/r/Synonym-/c/en/hole-0000","rel|label":"synonym","rel|relation":"/r/Synonym","rel|sentence":"","rel|source":"RG"},{"uid":"0x7a1fc","uri":"/c/en/up","label":"up","rel|edge_id":"/c/en/dump-/r/RelatedTo-/c/en/up-0000","rel|label":"related to","rel|relati